In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy import stats
from fitter import Fitter, get_common_distributions

In [9]:
class DataGenerator:
    def __init__(self,
                 num_blocks=50):

        self.num_blocks = num_blocks
        self.df = pd.read_excel('../data/블록-계획데이터(예제)_수정.xlsx')

        # 없는 데이터를 전부 제거한 데이터프레임 생성(이후 이 데이터프레임을 사용)
        self.df_revised = self.df.dropna(axis=0)
        idx_1 = self.df_revised[self.df_revised['취부팀_코드'] == 'XXX'].index
        self.df_revised = self.df_revised.drop(idx_1)
        idx_2 = self.df_revised[self.df_revised['용접팀_코드'] == 'XXX'].index
        self.df_revised = self.df_revised.drop(idx_2)
        idx_3 = self.df_revised[self.df_revised['stage_코드'] == 'XXX'].index
        self.df_revised = self.df_revised.drop(idx_3)
        idx_4 = self.df_revised[self.df_revised['정반_코드'] == 'XXXX'].index
        self.df_revised = self.df_revised.drop(idx_4)
        idx_5 = self.df_revised[self.df_revised['실적공기'] < 0].index
        self.df_revised = self.df_revised.drop(idx_5)
        self.df_revised.reset_index(inplace=True)

        # 공정 개수에 대한 데이터프레임
        # self.df_process_count = pd.DataFrame(self.df_revised['공종_명칭'].value_counts())
        # self.df_process_count.reset_index(inplace=True)
        # self.df_process_count['Proportion'] = self.df_process_count['count'] / self.df_process_count['count'].sum()

        # 그룹 개수에 대한 데이터프레임
        self.df_revised_for_group = self.df_revised.copy()
        group_list = []
        for i in range(self.df_revised.shape[0]):
            group_code = self.df_revised.loc[i, '선종_코드'] + '_' + self.df_revised.loc[i, '블록'][0]
            group_list.append(group_code)
        self.df_revised_for_group['선종_블록'] = group_list
        self.df_group_count = pd.DataFrame(self.df_revised_for_group['선종_블록'].value_counts())
        self.df_group_count.reset_index(inplace=True)
        self.df_group_count['Proportion'] = self.df_group_count['count'] / self.df_group_count['count'].sum()
        self.df_group_count['panel_count'] = 0
        self.df_group_count['curve_count'] = 0
        self.df_group_count['big_count'] = 0
        self.df_group_count['final_count'] = 0

        for code in self.df_group_count['선종_블록']:
            panel = 0
            curve = 0
            big = 0
            final = 0

            df_for_group = self.df_revised_for_group[self.df_revised_for_group['선종_블록'] == code]
            for process in df_for_group['공종_명칭']:
                if process == '평중조':
                    panel += 1
                elif process == '곡중조':
                    curve += 1
                elif process == '대조중조':
                    big += 1
                elif process == 'Final조립':
                    final += 1

            self.df_group_count.loc[self.df_group_count['선종_블록'] == code, 'panel_count'] = panel
            self.df_group_count.loc[self.df_group_count['선종_블록'] == code, 'curve_count'] = curve
            self.df_group_count.loc[self.df_group_count['선종_블록'] == code, 'big_count'] = big
            self.df_group_count.loc[self.df_group_count['선종_블록'] == code, 'final_count'] = final

        # 착수계획 개수에 대한 데이터프레임
        self.df_startplan_count = pd.DataFrame(self.df_revised['착수계획'].value_counts())
        self.df_startplan_count.sort_index(inplace = True)
        self.df_startplan_count.reset_index(inplace = True)
        date_range = pd.date_range(self.df_startplan_count.loc[0, '착수계획'], self.df_startplan_count.loc[self.df_startplan_count.shape[0] - 1, '착수계획'])
        date_range = list(date_range)
        changed_date = []

        for i in range(len(date_range)):
            for j in range(self.df_startplan_count.shape[0]):
                if self.df_startplan_count.loc[j, '착수계획'] == date_range[i]:
                    changed_date.append(i)

        self.df_startplan_count.insert(loc=1, column='changed_date', value=changed_date)
        self.df_startplan_count['Proportion'] = self.df_startplan_count['count'] / self.df_startplan_count['count'].sum()


    def generate_process(self, group_code):         # 공종 명칭 생성 함수, 공정이 나오는 비율에 맞춰서 데이터 생성
        df_process_group = self.df_group_count[self.df_group_count['선종_블록'] == group_code]
        # 각 count 값을 올바르게 추출
        count = df_process_group['count'].values[0]
        panel_proportion = df_process_group['panel_count'].values[0] / count
        curve_proportion = df_process_group['curve_count'].values[0] / count
        big_proportion = df_process_group['big_count'].values[0] / count
        final_proportion = df_process_group['final_count'].values[0] / count

        proportion_list = [panel_proportion, curve_proportion, big_proportion, final_proportion]

        process_type = np.random.choice(['평중조', '곡중조', '대조중조', 'Final조립'], p=proportion_list)
        return process_type


    def generate_group(self):       # 그룹을 선택한 후 선종과 블록 종류로 나누기 위한 함수
        # 그룹을 랜덤으로 선택->선종과 블록 타입으로 분리
        group = np.random.choice(self.df_group_count['선종_블록'], p=self.df_group_count['Proportion'])

        ship_type = group[0:2]
        block_type = group[-1]

        return (group, ship_type, block_type)


    def generate_start_date(self):

        start_date = np.random.choice(self.df_startplan_count['changed_date'], p=self.df_startplan_count['Proportion'])
        return start_date


    def generate_weight(self, group_code, process_type, length, breadth, height):
        if process_type == 'Final조립':
            df_revised_for_weight = self.df_revised_for_group[self.df_revised_for_group['선종_블록'] == group_code]
            df_revised_for_weight['LBH'] = df_revised_for_weight['L'] * df_revised_for_weight['H'] * df_revised_for_weight['B']

            # Final조립의 선형 피팅 모델 구현(group_code에 따라 달라짐)
            df_revised_for_final = df_revised_for_weight[df_revised_for_weight['공종_명칭'] == 'Final조립']
            x = df_revised_for_final['LBH'].to_numpy()
            x = x.reshape(-1, 1)
            y = df_revised_for_final['W'].to_numpy()
            reg = linear_model.LinearRegression()
            reg.fit(x, y)

            LBH_value = length * breadth * height

            weight = reg.coef_[0] * LBH_value + reg.intercept_
            weight = np.int64(weight)

            # 중조 공정(평중조, 곡중조, 대조중조)에 관해서는 0으로 결정
        else:
            weight = 0

        return weight

    def generate_property(self, group_code, property):
        df_code = self.df_revised_for_group[self.df_revised_for_group['선종_블록'] == group_code]
        df_for_fit = df_code[['선종_블록', property]]
        df_for_fit_count = pd.DataFrame(df_for_fit[property].value_counts())
        df_for_fit_count.reset_index(inplace=True)
        df_for_fit_count.sort_values(property, inplace=True)

        df_for_fit_count['Density'] = df_for_fit_count['count'] / df_for_fit_count['count'].sum()

        data = []
        for p, c in zip(df_for_fit_count[property], df_for_fit_count['count']):
            data.extend([p]*c)
        data = np.array(data)
        distributions_list = get_common_distributions()

        f = Fitter(data, distributions=distributions_list)
        f.fit()

        best_distribution = f.get_best()
        best_distribution_name = list(best_distribution.keys())[0]
        best_params = f.fitted_param[best_distribution_name]

        property_value = 0

        if best_distribution_name == 'cauchy':
            property_value = stats.cauchy.rvs(*best_params)
        elif best_distribution_name == 'expon':
            property_value = stats.expon.rvs(*best_params)
        elif best_distribution_name == 'gamma':
            property_value = stats.gamma.rvs(*best_params)
        elif best_distribution_name == 'norm':
            property_value = stats.norm.rvs(*best_params)
        elif best_distribution_name == 'exponpow':
            property_value = stats.exponpow.rvs(*best_params)
        elif best_distribution_name == 'lognorm':
            property_value = stats.lognorm.rvs(*best_params)
        elif best_distribution_name == 'powerlaw':
            property_value = stats.powerlaw.rvs(*best_params)
        elif best_distribution_name == 'reyleigh':
            property_value = stats.reyleigh.rvs(*best_params)
        elif best_distribution_name == 'uniform':
            property_value = stats.uniform.rvs(*best_params)

        if property_value > df_for_fit_count[property].max():
            property_value = df_for_fit_count[property].max()
        elif property_value < df_for_fit_count[property].min():
            property_value = df_for_fit_count[property].min()

        property_value = np.floor(property_value * 10) / 10

        return property_value


    def generate_duration(self, group_code, workload_H01, workload_H02):
        df_for_duration = self.df_revised_for_group[self.df_revised_for_group['선종_블록'] == group_code]

        x = df_for_duration[['H01', 'H02']]
        y = df_for_duration['계획공기']

        reg = linear_model.LinearRegression()
        reg.fit(x, y)

        duration = reg.coef_[0] * workload_H01 + reg.coef_[1] * workload_H02 + reg.intercept_
        duration = np.int64(duration)

        return duration



    def generate(self):
        columns = ["Block_Name", "Block_ID", "Process_Type", "Ship_Type", "Block_Type", "Start_Date", "Duration", "Workload_H01", "Workload_H02", "Weight", "Length", "Breadth", "Height"]

        df_blocks = []

        for j in range(self.num_blocks):
            # 블록 이름과 id 생성
            name = "J-%d" % j
            id = j

            # 선종과 블록 타입 생성
            group_results = self.generate_group()          # column에 포함되지는 않음
            group_code = group_results[0]
            ship_type = group_results[1]
            block_type = group_results[2]

            # 공종 명칭 생성
            process_type = self.generate_process(group_code)

            start_date = self.generate_start_date()

            length = self.generate_property(group_code, 'L')
            breadth = self.generate_property(group_code, 'B')
            height = self.generate_property(group_code, 'H')

            weight = self.generate_weight(group_code, process_type, length, breadth, height)

            workload_h01 = 263
            workload_h02 = 140

            duration = self.generate_duration(group_code, workload_h01, workload_h02)

            row = [name, id, process_type, ship_type, block_type, start_date, duration, workload_h01, workload_h02, weight, length, breadth, height]
            df_blocks.append(row)

        df_blocks = pd.DataFrame(df_blocks, columns=columns)


        return df_blocks

In [10]:
generator = DataGenerator()
df_blocks = generator.generate()
df_blocks

C:\Users\kimjh\AppData\Local\Temp\ipykernel_29204\1240641107.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_revised_for_weight['LBH'] = df_revised_for_weight['L'] * df_revised_for_weight['H'] * df_revised_for_weight['B']
C:\Users\kimjh\AppData\Local\Temp\ipykernel_29204\1240641107.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_revised_for_weight['LBH'] = df_revised_for_weight['L'] * df_revised_for_weight['H'] * df_revised_for_weight['B']
C:\Users\kimjh\AppData\Local\Temp\ipykernel_29204\

,Block_Name,Block_ID,Process_Type,Ship_Type,Block_Type,Start_Date,Duration,Workload_H01,Workload_H02,Weight,Length,Breadth,Height
0,J-0,0,곡중조,CN,E,175,6,263,140,0,15.5,3.0,7.9
1,J-1,1,곡중조,TK,E,223,7,263,140,0,13.7,5.0,7.2
2,J-2,2,대조중조,CN,B,218,6,263,140,0,13.3,9.7,1.1
3,J-3,3,Final조립,CN,S,347,7,263,140,81,19.2,3.5,5.7
4,J-4,4,Final조립,CN,S,252,7,263,140,122,19.0,13.5,6.5
5,J-5,5,Final조립,CN,F,700,8,263,140,73,19.1,9.0,4.9
6,J-6,6,Final조립,CN,A,356,7,263,140,82,15.4,9.5,4.9
7,J-7,7,곡중조,LN,E,469,6,263,140,0,16.6,5.0,5.3
8,J-8,8,평중조,PT,E,137,8,263,140,0,10.2,3.6,2.5
9,J-9,9,대조중조,CN,S,512,7,263,140,0,19.8,5.4,8.4
